# Setup

Sections of the notebook marked "Train" are used to train the model in the deployment pipeline.  All other cells will be excluded from the build of the model.

Train cells should include the binary saving out of the model (via joblib) as a "pickle" for reading in later

In [ ]:
# Common imports
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

# Assume a file data source
dataSource = 'File'

In [ ]:
# Enable intelliense
%config IPCompleter.greedy=True

%matplotlib inline
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(train_labels, train_pred):
    fig = plt.figure(figsize=(6,6))
    ax = plt.subplot()

    labels = list(train_labels['Class'].value_counts().index)
    desc_labels = ['Legit', 'Fraud']
    #print(labels)

    confusion = confusion_matrix(train_labels, train_pred, labels=labels)
    ax.matshow(np.log(confusion + 1.001))

    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))

    ax.set_xticklabels(desc_labels, rotation=90);
    ax.set_yticklabels(desc_labels);

    for i in range(len(labels)):
        for j in range(len(labels)):        
            ax.text(j, i, confusion[i,j], va='center', ha='center').set_backgroundcolor("white")

    plt.xlabel('Predicted')    
    plt.ylabel('Actual')
    
    return fig

# Select Input: "Kernel Restart & Run All" will block until the "Proceed" button is clicked after input selection.

In [ ]:
# you need to 'pip3 install ipython_blocking' for blocking to work
import ipywidgets as widgets
from IPython.display import display
import ipython_blocking

selectw = widgets.RadioButtons(
    options=['File', 'Spark', 'VDB'],
    description='Select Input:',
    disabled=False
)

button = widgets.Button(description='Proceed',layout= widgets.Layout(border='1px solid black'))

box = widgets.VBox(children=[selectw, button], layout=widgets.Layout(border='solid', width='240px', height='160px'))
display(box)

# Use Magic to Block Until "Proceed" Button Clicked

In [ ]:
%blockrun button
dataSource = selectw.value

# Read Data Based On Selection

In [ ]:
if dataSource == 'Spark':
    print("Spark input not yet implemented. Switching to use: File")
    dataSource = 'File'

In [ ]:
if dataSource == 'VDB':
    import jaydebeapi
    
    # The Vitual Database or VDB was created using Red Hat Data Virtualization.
    # The VDB provides a virtual view of credit card fraud data consolidated from 2 sources:
    # 1. MySQL (historical data from data warehouse)
    # 2. A CSV file (recent data)
    
    # set up VDB access
    # ****************** replace IP address with your VDB service cluster IP
    url = "jdbc:teiid:csvrdbmsdb.1@mm://172.30.126.110:31000"
    driver = "org.teiid.jdbc.TeiidDriver"
    user = "user"
    password = "mypassword"
    jarfile = "jdbc/teiid-9.0.6-jdbc.jar"
    conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
    SQL_Query = "select * from CreditFraud"

    df = pd.read_sql_query(SQL_Query, conn)
    conn.close()

    # rename specific columns
    df.rename(columns={'Time_':'Time', 'Class_':'Class'}, inplace=True)
    
    print("Data loaded using VDB.")


In [ ]:
if dataSource == 'File':

    print("Reading csv data")
    df = pd.read_csv ("data/creditcard.csv")
    # rename specific columns
    df.rename(columns={'Unnamed: 0':'Id'}, inplace=True)
    
    print("Data loaded using File.")


In [ ]:
### Display some info about the data
print("Total number of credit card transaction rows: %d" % df.shape[0])

### Check the total number of rows with fraud detected
print("Total number of rows with fraud: %d" % df[(df['Class']==1)].shape[0])

### Rough check if data columns have consistent datatype (not Object)
print(df.dtypes)

# Train Sklearn Random Forest Model

In [ ]:
# shuffle the rows
df = df.sample(frac=1).reset_index()

#Order the credit card transaction by transaction time
#df.sort_values(by=['Time'])

#number of rows in the dataset
n_samples = df.shape[0]
print("Samples: %d" % n_samples)

#Split into train and test
train_size = 0.75

train_limit = int(n_samples * train_size)
df_train = df.iloc[:train_limit]
df_test = df.iloc[train_limit:]


# Initial Model Creation

In [ ]:
import joblib

print('Training sample size: %d' % df_train.shape[0])
frauds_in_train_tx = df_train[df_train.Class == 1].shape[0]
print('Frauds transactions in training sample: %f' % (frauds_in_train_tx / train_limit))
print('Testing sample size: %d' % df_test.shape[0])
frauds_in_test_tx = df_test[df_test.Class == 1].shape[0]
print('Frauds transactions in testing sample: %f' % (frauds_in_test_tx / (n_samples - train_limit)))
      
features=['V25', 'V28', 'V13', 'V24', 'V26', 'V22', 'V15']
non_features = [i for i in df_train.columns if i not in features]

#Define features and target variables for convenience.
drop_columns = ['Id', 'Time', 'Class', 'V16','V4','V10','V11','V12','V14','V17', 'V7', 'V9', 'V2']
select_column=['Class']

#Create Train Datasets
features_train = df_train.drop(non_features, axis=1)
target_train = df_train.loc[:, select_column]
#print("*Data Columns in features_train:")
#print(features_train.columns)
#print("*Data Columns in target_train:")
#print(target_train.columns)


#Create Test Datasets
features_test = df_test.drop(non_features, axis=1)
target_test = df_test.loc[:, select_column]

#Create a RandomForest Classifier mode
model = RandomForestClassifier(n_estimators=10, max_depth=5, n_jobs=10, class_weight='balanced')

model.fit(features_train, target_train.values.ravel())

pred_train = model.predict(features_train)
pred_test = model.predict(features_test)

pred_train_prob = model.predict_proba(features_train)
pred_test_prob = model.predict_proba(features_test)

print("Number of features: %d" % len(model.feature_importances_))

  
from sklearn.metrics import accuracy_score
print("Model accuracy: %5.4f" % accuracy_score(target_test, pred_test))

#save mode in filesystem
joblib.dump(model, 'model.pkl') 

# Plot the Confusion Matrix

In [ ]:
_=plot_confusion_matrix(target_train, model.predict(features_train))

_=plot_confusion_matrix(target_test, model.predict(features_test))

# Check the Important Features

In [ ]:
import operator

feat_imp = sorted(zip(features_train.columns, model.feature_importances_), key=operator.itemgetter(1), reverse=True)
X = [i[0] for i in feat_imp]
Y = [i[1] for i in feat_imp]
plt.figure(figsize=(10.,10.))

# plt.plot([i[0] for i in feat_imp], [i[1] for i in feat_imp], 'p-')
# sorts the X axis by the label name resulting in a weird-looking plot
plt.plot(Y, 'p-')
_ = plt.xticks(range(len(Y)), X, rotation=90)

##### Re-create the model with Important Features

# Alternatively, Perform a Correlation Analysis 
We will come to a similar conclusion on which features to include in the model: those with a low correlation coefficient should be excluded to avoid overfitting.
The graph has the features with the highest correlation coefficent plotted first. Class is to be excluded as it is the target for the correlation.
This shows the importance of data analysis.

In [ ]:
corr = df.loc[:, 'Class']

dfc = df.corrwith(corr).to_frame()
# use a temporary column to sort and then discard the column
dfc = (dfc.assign(A=abs(dfc))
    .sort_values(['A'],ascending=False)
    .drop('A', 1))
#print(dfc)
dfc.plot.bar(
        figsize = (20, 10), title = "Correlation with Class", fontsize = 15,
        rot = 45, grid = True)

# Re-create the model with Important Features

In [ ]:
import joblib

#Define features and target variables.

# Choose either the features definition based on important features or correlation

## From the important features graph we only want seven important features:
##   V3,V4,V10,V11,V12,V14,V17
features = ['V16','V4','V10','V11','V12','V14','V17']

## using the correllation graph, select the top seven important features:
## V14,V17,V3,V12,V10,V16,V11
#features = ['V14','V17','V3','V12','V10','V16','V11']

non_features = [i for i in df_train.columns if i not in features]
class_column=['Class']


features_train = df_train.drop(non_features, axis=1)
target_train = df_train.loc[:, "Class"]

features_test = df_test.drop(non_features, axis=1)
target_test = df_test.loc[:, "Class"]
print("feature_test columns:")
print(features_test.columns)

model = RandomForestClassifier(n_estimators=1000, max_depth=20, n_jobs=10, class_weight='balanced')

model.fit(features_train, target_train.values.ravel())

pred_train = model.predict(features_train)
pred_test = model.predict(features_test)

pred_train_prob = model.predict_proba(features_train)
pred_test_prob = model.predict_proba(features_test)

print("Number of features")
print(len(model.feature_importances_))
  
from sklearn.metrics import accuracy_score
print("Model accuracy: %5.4f" % accuracy_score(target_test, pred_test))
 
#save mode in filesystem
joblib.dump(model, 'model.pkl') 

# Re-plot the Confusion Matrix

In [ ]:
_ = plot_confusion_matrix(target_train.to_frame(), model.predict(features_train))

_ = plot_confusion_matrix(target_test.to_frame(), model.predict(features_test))

# Test the New Model

In [ ]:
import time


fraudTest = df_test.loc[df_test['Class']== 1]
notFraudTest = df_test.loc[df_test['Class']== 0]

fraudTestFeatures = fraudTest.drop(columns=non_features)
notFraudTestFeatures = notFraudTest.drop(columns=non_features)

for index, row in fraudTestFeatures.iterrows():
    data = row
    rowdf = pd.DataFrame([data.tolist()], columns = features)
    print(model.predict(rowdf))
    # time.sleep(2)


# New Model output

This section can be used for testing the model once it is deployed.  It mimics the input and output expected from the [Seldon model](https://github.com/kalefranz/seldon-python-microservice/blob/ee6b49f7670ed8ecb04912400333db6781ee4110/seldon_microservice/model_microservice.py) that ultimately wraps the deployment (see below).  This is tagged to be part of the training output so that the results here can be compared to what's deployed for debugging purposes.

Results should be *similar* to what it output here if the model is (re)trained in the pipeline.  This is due to the testing data being shuffled (see above code block) and also due to the nature of the training process for the RandomForestClassifier.

If, however, the model that is saved out above is the exact one that goes into the pipeline, then the results should be identical



In [ ]:
def get_class_names(user_model,n_targets):
    if hasattr(user_model,"class_names"):
        return user_model.class_names
    else:
        return ["t:{}".format(i) for i in range(n_targets)]
    
def array_to_rest_datadef(array,names):
    datadef = {"names":names}
    datadef["ndarray"] = array.tolist()
    return datadef

loadedModel = joblib.load('model.pkl')
print ("Loaded model is: ", loadedModel)

loadedModel.class_names = ['V3','V4','V10','V11','V12','V14','V17']
X=[[-4.304597,  4.732795, -2.447469,  2.101344, -4.609628, -6.079337,  6.739384]]
Y=np.array(X)

print ("X is: ", X)
print ("Prediction from model for X is: ", loadedModel.predict_proba(X))
print ("Y (numpy array) is: ", Y)
print ("Prediction from model for Y is: ", loadedModel.predict_proba(Y))
pred = np.array(loadedModel.predict_proba(Y))
print ("Prediction for Y (in Seldon return format) is: ", pred)
print ("pred.shape is: ", pred.shape)
print ("Output from model for predict on Y is expected to look like this: ",  array_to_rest_datadef(pred, get_class_names(loadedModel, pred.shape[1])))

# Seldon Deployment

This section will be extracted to represent the model being loaded in a format that is compatible with Seldon Core as outlined [in the documentation](https://docs.seldon.io/projects/seldon-core/en/v1.2.0/python/python_component.html)

In [ ]:
import joblib

class Wrapper(object):

    def __init__(self):
        self.model = joblib.load('model.pkl')
        self.class_names = ['V3','V4','V10','V11','V12','V14','V17']
        print ("Model loaded.  Initially is:", self.model)

    def predict(self,X,features_names):
        print ("Loaded model is: ", self.model)
        print ("X is:")
        print (X)
        print ("features_names is: ")
        print (features_names)
        prediction = self.model.predict_proba(X)
        print ("output of predict_proba is: ", prediction)
        return prediction

# Git Helper

This section is to aid in checking in changes to the notebook.  Uncomment and run the commands below to checkin all changes made to the notebook

In [ ]:
# !cd ..
# !git pull origin
# !git status
# !git commit -am"New Deploy"
# !git push origin